In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install sentence-transformers
!pip install transformers

In [31]:
!pip install numpy

In [83]:
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, PyPDFDirectoryLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.embeddings import OllamaEmbeddings, HuggingFaceEmbeddings, HuggingFaceBgeEmbeddings
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage
from langchain.chains import (
    RetrievalQA,
    create_retrieval_chain,
    create_history_aware_retriever,
    ConversationChain,
)
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory
import bs4
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
import os
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate


In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install transformers

In [11]:
from langchain_community.llms import Ollama

In [13]:
from langchain_community.vectorstores import Chroma, FAISS

In [17]:
from langchain_community import embeddings

In [12]:
model = Ollama(model="llama2")

In [19]:
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
huggingface_embeddings.embed_query

<bound method HuggingFaceBgeEmbeddings.embed_query of HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en-v1.5', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='')>

In [63]:
loader=PyPDFDirectoryLoader("./Mental")

documents=loader.load()
print(documents)
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

[Document(page_content='UNDERSTANDING MENTAL HEALTH\nWHAT IS MENTAL HEALTH?\nOur mental health directly influences how we think, feel and act: it also affects our physical health. Work, in fact, is actually one of \nthe best things for protecting our mental health, but it can also adversely affect it. \nGood mental health and well-being is not an on-off \nexperience. We can all have days, weeks or months \nwhere we feel resilient, strong and optimistic, \nregardless of events or situations. Often that can \nbe mixed with or shift to a very different set of \nthoughts, feelings and behaviours; or not feeling \nresilient and optimistic in just one or two areas of \nour life.  For about twenty-five per cent of us, that \nmay shift to having a significant impact on how \nwe think, feel and act in many parts of our lives, \nincluding relationships, experiences at work, sense \nof connection to peer groups and our personal sense \nof worth, physical health and motivation. This could \nlead t

Document(page_content='UNDERSTANDING MENTAL HEALTH\nWHAT IS MENTAL HEALTH?\nOur mental health directly influences how we think, feel and act: it also affects our physical health. Work, in fact, is actually one of \nthe best things for protecting our mental health, but it can also adversely affect it. \nGood mental health and well-being is not an on-off \nexperience. We can all have days, weeks or months \nwhere we feel resilient, strong and optimistic, \nregardless of events or situations. Often that can \nbe mixed with or shift to a very different set of \nthoughts, feelings and behaviours; or not feeling \nresilient and optimistic in just one or two areas of \nour life.  For about twenty-five per cent of us, that \nmay shift to having a significant impact on how \nwe think, feel and act in many parts of our lives, \nincluding relationships, experiences at work, sense \nof connection to peer groups and our personal sense \nof worth, physical health and motivation. This could', metadat

In [21]:
# urls = 'https://www.who.int/news-room/fact-sheets/detail/mental-health-strengthening-our-response'
# urls_list = urls.split("\n")
# docs = [WebBaseLoader(url).load() for url in urls_list]
# docs_list = [item for sublist in docs for item in sublist]

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
#                                                 chunk_overlap=50,
#                                                 length_function=len)

# all_splits = text_splitter.split_documents(docs_list)


In [64]:
final_documents[0]

Document(page_content='UNDERSTANDING MENTAL HEALTH\nWHAT IS MENTAL HEALTH?\nOur mental health directly influences how we think, feel and act: it also affects our physical health. Work, in fact, is actually one of \nthe best things for protecting our mental health, but it can also adversely affect it. \nGood mental health and well-being is not an on-off \nexperience. We can all have days, weeks or months \nwhere we feel resilient, strong and optimistic, \nregardless of events or situations. Often that can \nbe mixed with or shift to a very different set of \nthoughts, feelings and behaviours; or not feeling \nresilient and optimistic in just one or two areas of \nour life.  For about twenty-five per cent of us, that \nmay shift to having a significant impact on how \nwe think, feel and act in many parts of our lives, \nincluding relationships, experiences at work, sense \nof connection to peer groups and our personal sense \nof worth, physical health and motivation. This could', metadat

In [ ]:
import  numpy as np
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

In [66]:
conversation = ConversationChain(
    llm=model, verbose=True, memory=ConversationBufferMemory()
)

In [74]:
template = """
            You are talking to a 12 year old boy who has mental trauma.
            You are a professional expertise with acurate knowledge on mental health.
            Your job is to use the following context to answer questions about a how to take care of the anxious person.
            The context provided are from academic researches and are highly reliable.
            The context is in a particular format but you should NOT mimic the style.
            Always answer the question using normal language with emotion and sentiment.
            Context: {context}

            """

In [75]:
vectorstore=FAISS.from_documents(final_documents[:200],huggingface_embeddings)

In [ ]:
query="i am depressed"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

In [90]:


prompt_template = """
            You are a professional expertise with acurate knowledge on mental health.
            Your job is to use the following context to answer questions about a how to take care of a mental patient.
            The context provided are from academic researches and are highly reliable.
            Try to stick to the context as much as possible.
            The context is in a particular format but you should NOT mimic the style.
            Always answer the question using normal language with emotion and sentiment.
            The user are people with possible mental conditions. So you can try to paraphrase the answer to make it more emotionally resonant to the user.
            At the end of your answer, include only the file name website link and page number of the source. For example: "Source: some_file.pdf,website.com,page 5".
            Context: {context}
            Question: {question}
            Reply :
            """



    # prompt=PromptTemplate(template=template,input_variables=["context","question"])
    # prompt_string = prompt.render(context="", question=question)



In [91]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7af09d5f4af0> search_kwargs={'k': 3}


In [92]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [94]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [95]:
query=""" i have depression """

In [97]:
result = retrievalQA.invoke({"query": query})
print(result['result'])

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7af04a54c6d0>: Failed to establish a new connection: [Errno 111] Connection refused'))